In [ ]:
"""
k-Nearest Neighbors (kNN) Classifier - Clean Implementation
CS231n Lecture 2: Image Classification

Author: Hamza
Date: November 2024
"""

import numpy as np
import matplotlib.pyplot as plt
from collections import Counter


# ============================================================================
# Part 1: Basic Nearest Neighbor Classifier
# ============================================================================

class NearestNeighbor:
    """
    Nearest Neighbor classifier using L1 (Manhattan) distance.
    
    This is the simplest form of kNN with k=1.
    
    Algorithm:
        1. Train: Memorize all training data (O(1) time)
        2. Predict: For each test example, find closest training example
    
    Time Complexity:
        - Train: O(1) - just store the data
        - Predict: O(N*D) per test example
            N = number of training examples
            D = dimensionality of each example
    
    Space Complexity: O(N*D) - store all training data
    """
    
    def __init__(self):
        """Initialize the classifier."""
        self.Xtr = None
        self.ytr = None
    
    def train(self, X, y):
        """
        Train the classifier by memorizing training data.
        
        Args:
            X: Training data (N, D) where N = number of examples, D = features
            y: Training labels (N,) - one label per example
        
        Note: In kNN, "training" just means storing the data.
              There are no parameters to learn!
        """
        self.Xtr = X
        self.ytr = y
        print(f"✓ Trained on {X.shape[0]} examples with {X.shape[1]} features each")
    
    def predict(self, X):
        """
        Predict labels for test data using nearest neighbor.
        
        Args:
            X: Test data (M, D) where M = number of test examples
        
        Returns:
            y_pred: Predicted labels (M,)
        
        Algorithm:
            For each test example:
                1. Compute L1 distance to all training examples
                2. Find the training example with minimum distance
                3. Return its label
        """
        num_test = X.shape[0]
        y_pred = np.zeros(num_test, dtype=self.ytr.dtype)
        
        print(f"Predicting {num_test} test examples...")
        for i in range(num_test):
            # Compute L1 distance: |x_test - x_train| summed over all features
            # Shape: (N,) where N = number of training examples
            distances = np.sum(np.abs(self.Xtr - X[i, :]), axis=1)
            
            # Find index of minimum distance
            min_index = np.argmin(distances)
            
            # Predict the label of the nearest neighbor
            y_pred[i] = self.ytr[min_index]
            
            # Progress indicator
            if (i + 1) % 100 == 0:
                print(f"  Processed {i + 1}/{num_test} examples")
        
        return y_pred


# ============================================================================
# Part 2: k-Nearest Neighbors Classifier (Generalized)
# ============================================================================

class KNearestNeighbor:
    """
    k-Nearest Neighbors classifier with configurable k and distance metric.
    
    Improvements over basic NN:
        - Support for k > 1 (voting among multiple neighbors)
        - Choice of distance metric (L1 or L2)
        - Cleaner API
    """
    
    def __init__(self, k=1, distance_metric='L1'):
        """
        Initialize kNN classifier.
        
        Args:
            k: Number of neighbors to consider (default: 1)
            distance_metric: 'L1' (Manhattan) or 'L2' (Euclidean)
        """
        self.k = k
        self.distance_metric = distance_metric
        self.Xtr = None
        self.ytr = None
    
    def train(self, X, y):
        """Memorize training data."""
        self.Xtr = X
        self.ytr = y
        print(f"✓ Trained kNN (k={self.k}, metric={self.distance_metric})")
        print(f"  Dataset: {X.shape[0]} examples, {X.shape[1]} features")
    
    def _compute_distances(self, X):
        """
        Compute distances between test examples and all training examples.
        
        Args:
            X: Test data (M, D)
        
        Returns:
            distances: (M, N) where distances[i, j] = distance from X[i] to Xtr[j]
        """
        num_test = X.shape[0]
        num_train = self.Xtr.shape[0]
        distances = np.zeros((num_test, num_train))
        
        for i in range(num_test):
            if self.distance_metric == 'L1':
                # L1 distance: Σ|x - y|
                distances[i, :] = np.sum(np.abs(self.Xtr - X[i, :]), axis=1)
            elif self.distance_metric == 'L2':
                # L2 distance: √(Σ(x - y)²)
                distances[i, :] = np.sqrt(np.sum((self.Xtr - X[i, :]) ** 2, axis=1))
            else:
                raise ValueError(f"Unknown distance metric: {self.distance_metric}")
        
        return distances
    
    def predict(self, X):
        """
        Predict labels using k-nearest neighbors voting.
        
        Args:
            X: Test data (M, D)
        
        Returns:
            y_pred: Predicted labels (M,)
        """
        # Compute all distances at once
        distances = self._compute_distances(X)
        
        num_test = X.shape[0]
        y_pred = np.zeros(num_test, dtype=self.ytr.dtype)
        
        for i in range(num_test):
            # Find indices of k nearest neighbors
            # np.argsort returns indices that would sort the array
            # [:self.k] takes the first k (smallest distances)
            k_nearest_indices = np.argsort(distances[i, :])[:self.k]
            
            # Get labels of k nearest neighbors
            k_nearest_labels = self.ytr[k_nearest_indices]
            
            # Vote: most common label wins
            # Counter counts occurrences, most_common(1) returns [(label, count)]
            y_pred[i] = Counter(k_nearest_labels).most_common(1)[0][0]
        
        return y_pred
    
    def score(self, X, y):
        """
        Compute accuracy on test data.
        
        Args:
            X: Test data (M, D)
            y: True labels (M,)
        
        Returns:
            accuracy: Fraction of correct predictions
        """
        y_pred = self.predict(X)
        accuracy = np.mean(y_pred == y)
        return accuracy


# ============================================================================
# Part 3: Demonstration & Testing
# ============================================================================

def demo_basic_nn():
    """Demonstrate basic Nearest Neighbor classifier."""
    print("=" * 70)
    print("DEMO 1: Basic Nearest Neighbor (k=1)")
    print("=" * 70)
    
    # Generate synthetic data
    np.random.seed(42)
    
    # Class 0: Points near (0, 0)
    X0 = np.random.randn(50, 2) * 0.5
    y0 = np.zeros(50, dtype=int)
    
    # Class 1: Points near (3, 3)
    X1 = np.random.randn(50, 2) * 0.5 + 3
    y1 = np.ones(50, dtype=int)
    
    # Combine
    X_train = np.vstack([X0, X1])
    y_train = np.hstack([y0, y1])
    
    # Create test set
    X_test = np.array([[0.5, 0.5], [2.5, 2.5], [3.5, 3.5], [-0.5, -0.5]])
    y_test = np.array([0, 1, 1, 0])  # True labels
    
    # Train and predict
    nn = NearestNeighbor()
    nn.train(X_train, y_train)
    y_pred = nn.predict(X_test)
    
    # Show results
    print("\nResults:")
    for i in range(len(X_test)):
        print(f"  Test point {X_test[i]}: Predicted={y_pred[i]}, True={y_test[i]}")
    
    accuracy = np.mean(y_pred == y_test)
    print(f"\nAccuracy: {accuracy * 100:.1f}%")


def demo_knn_voting():
    """Demonstrate k-NN with voting."""
    print("\n" + "=" * 70)
    print("DEMO 2: k-Nearest Neighbors with Voting")
    print("=" * 70)
    
    # Same data as before
    np.random.seed(42)
    X0 = np.random.randn(50, 2) * 0.5
    y0 = np.zeros(50, dtype=int)
    X1 = np.random.randn(50, 2) * 0.5 + 3
    y1 = np.ones(50, dtype=int)
    X_train = np.vstack([X0, X1])
    y_train = np.hstack([y0, y1])
    
    # Test different k values
    X_test = np.array([[1.5, 1.5]])  # Boundary point
    
    for k in [1, 3, 5, 10]:
        knn = KNearestNeighbor(k=k, distance_metric='L2')
        knn.train(X_train, y_train)
        y_pred = knn.predict(X_test)
        print(f"k={k:2d}: Predicted class = {y_pred[0]}")


def compare_distance_metrics():
    """Compare L1 vs L2 distance."""
    print("\n" + "=" * 70)
    print("DEMO 3: L1 vs L2 Distance Metrics")
    print("=" * 70)
    
    # Generate data
    np.random.seed(42)
    X_train = np.random.randn(200, 10)  # 200 examples, 10 features
    y_train = np.random.randint(0, 3, size=200)  # 3 classes
    
    X_test = np.random.randn(50, 10)
    y_test = np.random.randint(0, 3, size=50)
    
    # Compare metrics
    for metric in ['L1', 'L2']:
        knn = KNearestNeighbor(k=5, distance_metric=metric)
        knn.train(X_train, y_train)
        accuracy = knn.score(X_test, y_test)
        print(f"{metric} distance: Accuracy = {accuracy * 100:.2f}%")


# ============================================================================
# Main Execution
# ============================================================================

if __name__ == "__main__":
    # Run all demos
    demo_basic_nn()
    demo_knn_voting()
    compare_distance_metrics()
    
    print("\n" + "=" * 70)
    print("✓ All demos completed successfully!")
    print("=" * 70)




